In [19]:
import argparse
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
from keras.preprocessing.image import load_img,img_to_array
import model.create_dataset as create_dataset
from model.utils import Params
from model.model_fn import model_fn
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [21]:
model_dir = 'experiments/base_model'
#filename = 'test/Pose1/image_193.jpg'

In [22]:
def _parse_function(filename, image_size, channels):
    # Read an image from a file
    # Decode it into a dense vector
    # Resize it to fixed shape
    # Reshape it to 1 dimensional tensor
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=channels)
    image_resized = tf.image.resize_images(image_decoded, [image_size, image_size])
    features = tf.reshape(image_resized, [image_size*image_size*channels])
    features_normalized = features / 255.0
    return features_normalized

In [23]:
def _get_dataset(filename, params):

    # A tensor of only one filename
    filename_tensor = tf.constant([filename])

    # Load necessary params from config file
    image_size = params.image_size
    channels = 3 if params.rgb else 1

    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices(filename_tensor)
    dataset = dataset.map(lambda filename: _parse_function(filename, image_size, channels))
    dataset = dataset.batch(1)

    return dataset

In [24]:
def _get_embeddings(filename, estimator, params):
        
    # Compute embeddings for an image file
    tf.logging.info("Predicting on "+filename)

    predictions = estimator.predict(lambda: _get_dataset(filename, params))

    embeddings = np.zeros((1, params.embedding_size))
    for i, p in enumerate(predictions):
        embeddings[i] = p['embeddings']

    tf.logging.info("Image embedding shape: {}".format(embeddings.shape))

    return embeddings

In [ ]:
# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = Params(json_path)

tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)

# Define the model
tf.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed=230,
                                model_dir=model_dir,
                                save_summary_steps=params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

# Load landmark embeddings from disk
data = pd.read_csv(os.path.join(model_dir, "landmarks/embeddings.txt"), sep="\t")
data['embeddings'] = data['embeddings'].apply(lambda x : list(map(float, x.split(','))))
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose1/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6) 

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose1'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose2/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose2'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose3/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose3'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose4/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose4'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose5/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose5'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose6/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose6'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))

In [ ]:
count = 0
total_image_count = 0
for filename in glob.glob('test/Pose7/*.jpg'):
    #print(filename)
    img = load_img(filename, target_size=(224, 224))
    # Get embeddings
    embeddings = _get_embeddings(filename, estimator, params)
    embeddings = np.round(embeddings, 6)
    #print(embeddings)
    # Compute image distance to all landmarks
    data['dist'] = data['embeddings'].apply(lambda x : np.linalg.norm(embeddings - np.array(x)))
    data['dist'] = np.round(data['dist'], 6)

    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()

    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("INFO:Image name is: {}".format(filename))
    print("INFO:Predicted class: {}".format(label))
    print("INFO:Distance: {}".format(dist))
    plt.imshow(img)
    plt.show()
    if label == ('Pose7'):
        count+=1
    total_image_count+=1
print("Total count of correct predictions is {}".format(count))
print("Total image count is {}".format(total_image_count))